In [217]:
#2021 05 17, 18, 19 Создание отчета план - факт по БДДС. Промтех. Вологорад. ЗП

# в этом файле (C:\Users\Smolenyshev_OV\Documents\Python\ODDS\pt_odds_2021_v3.ipynb) много тестировал и писал различные варианты решения отчета по плвн-факт. Надо оставить для последующего использования

# делалю только по зп

#2021 05 24

# 2021 09 06 делаю уже из источника MSSQL 

# 2021 09 07 удалил лишнее из варианта 2. Иду по пути выделения, группировки по полю YM, соединения баз ...

# 2021 09 08 убрал лишнее. Перешел с where на .loc при условном формировании столбца. Оставил только merge. Обнаружил что названия статей отличаются ...
# надо переходить на агрегированные статьи и еще разобраться как автоматически отбирать данные по последнему тукущему дню месяца. Получилось. Только с периодом отбора на до разобраться. Да и еще может быть 'YM' добавить.

# 2021 09 09 Оставил только merge outer
# добавляю и пробую табл с YM
# осталось только доума довести табл в html - один знак после запятой

# 2021 09 10 еще надо сделать вывод в PDF - отставил пока создание PDF не так все просто. График добавляю

# 2021 09 11 Огниково добавляю графики
# 2021 09 16  добавляю текст в письмо с цифрами

# pt_odds_2.py тут продолжил вернулся

# 2021 10 20 соеденить CF и ОДДС в одном отчете

# 2023-08-29 Фр редакция для Git

In [218]:
import os
os.getcwd()

'c:\\Reports'

In [219]:
# os.chdir('\\\\VPROMFS01\\Share\\ФинДир\\Python\\ODDS')
# os.getcwd()
# os.chdir('c:\\Reports')

# \\172.16.50.6\Smolenyshev_OV\Уровень_1\Reports

os.chdir('c:\\Reports')
os.getcwd()

'c:\\Reports'

In [220]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, date, timedelta
import xlsxwriter
import openpyxl
import re

import pymssql

from copy import deepcopy

import datetime as dt


In [221]:
import matplotlib.pyplot as plt
import seaborn as sns

In [222]:
pd.set_option('display.float_format', '{:.2f}'.format)
# pd.set_option('display.max_rows', 100)

In [223]:
# подлючение к базе mssql
conn = pymssql.connect(server='vprom1c', user='**', password='***', database='Upload', charset='cp1251')
cursor = conn.cursor()  

cursor.execute('SELECT * FROM uld_mtpromtehbuh.details')  

row = cursor.fetchone()

desc = cursor.description
row = cursor.fetchall()

# columns = ['id', 'Depatment', 'Doc_id', 'date', 'SD', 'PordDt', 'SKD1', 'SKD1_cod', 'SKD1_tip', 'SKD2', 'SKD2_cod', 'SKD2_tip', 'SKD3', 'SKD3_cod', 'SKD3_tip', 'SK', 'PordKt','SKK1', 'SKK1_cod', 'SKK1_tip', 'SKK2', 'SKK2_cod', 'SKK2_tip', 'SKK3', 'SKK3_cod', 'SKK3_tip', 'Summa', 'Valuta', 'ValSumma', 'Kol']

# вывод названия столбцов по новому:
columns = [item[0] for item in cursor.description]


df0 = pd.DataFrame(row, columns=columns)

# 2021 10 20 загружает 8,2 сек

In [224]:
df0.tail()

,id,Depatment,Doc_id,date,SD,PodrDt,SKD1,SKD1_cod,SKD1_tip,SKD2,...,SKK2,SKK2_cod,SKK2_tip,SKK3,SKK3_cod,SKK3_tip,Summa,Valuta,ValSumma,Kol
164489,2208251,"ООО ""ПРОМТЕХ""",5ac319e6-30d7-11ec-bda3-00155d1e1e08,2021-10-20,91.02,Коммерческий отдел,91.1.04 Прочие внереализационные доходы,5.1.4,Прочие доходы и расходы,КОЛОННА АБСОРБЦИОННАЯ,...,Склад №2,,Склады,,,,0.00,,0.00,1.00
164490,2208252,"ООО ""ПРОМТЕХ""",5ac319e6-30d7-11ec-bda3-00155d1e1e08,2021-10-20,91.02,Коммерческий отдел,91.1.04 Прочие внереализационные доходы,5.1.4,Прочие доходы и расходы,,...,,,,,,,84333.33,,0.00,0.00
164491,2208253,"ООО ""ПРОМТЕХ""",a304c2a0-30d8-11ec-bda3-00155d1e1e08,2021-10-20,91.02,Коммерческий отдел,91.1.04 Прочие внереализационные доходы,5.1.4,Прочие доходы и расходы,,...,,,,,,,5000.00,,0.00,0.00
164492,2208254,"ООО ""ПРОМТЕХ""",a304c2a0-30d8-11ec-bda3-00155d1e1e08,2021-10-20,91.02,Коммерческий отдел,91.1.04 Прочие внереализационные доходы,5.1.4,Прочие доходы и расходы,934088 Ротор кк2010.01.100 (шт),...,Основной,,Склады,,,,0.00,,0.00,1.00
164493,2208255,"ООО ""ПРОМТЕХ""",a304c2a0-30d8-11ec-bda3-00155d1e1e08,2021-10-20,62.01,Коммерческий отдел,УРАЛЭНЕРГОМАШ ООО,7412008393,Контрагенты,Продажа ротора № 17/343 от 13.10.2021,...,934088 Ротор кк2010.01.100 (шт),ЦБ-00003276,Реализуемые активы,,,,30000.00,,0.00,1.00


In [225]:
print(df0['Depatment'].unique())

#Создаю дата фрейм только по Промтех

df0 = df0[df0['Depatment'] == 'ООО "ПРОМТЕХ"']

['ООО "ПРОМТЕХ"' 'ООО "ДЖИ ТИ ЭМ 1"' 'ООО "ВОЛГОГРАД ПОЛИМЕР"'
 'ООО "ПАТТЕРН"' 'ООО «ГХК»' 'ООО "ВПМ"' 'ООО "ПТ - МАЛЫЕ ПРОИЗВОДСТВА"']


In [226]:
# Отбор статей движения ден средств
# df = df0[(df0['SKD1_tip']=='Статьи движения денежных средств') | (df0['SKD2_tip']=='Статьи движения денежных средств') | (df0['SKK1_tip']=='Статьи движения денежных средств') | (df0['SKK2_tip']=='Статьи движения денежных средств')]

# отбор только по счетам

df = df0[(df0['SD']=='50') | (df0['SD']=='51') | (df0['SD']=='52') | (df0['SK']=='50') | (df0['SK']=='51') | (df0['SK']=='52')]

In [227]:
#Столбец со статьями ДС

# df['Статья'] = np.where(df['SD']=='50', df['SKD1'],\
#                np.where(df['SD']=='51', df['SKD2'],\
#                np.where(df['SD']=='52', df['SKD2'],\
#                np.where(df['SK']=='50', df['SKK1'],\
#                np.where(df['SK']=='51', df['SKK2'],\
#                np.where(df['SK']=='50', df['SKK2'], 
#                ''))))))


# перешел на .lov
df['Статья'] = ''

df.loc[df['SD']=='50', 'Статья'] = df['SKD1']
df.loc[df['SD']=='51', 'Статья'] = df['SKD2']
df.loc[df['SD']=='52', 'Статья'] = df['SKD2']
df.loc[df['SK']=='50', 'Статья'] = df['SKK1']
df.loc[df['SK']=='51', 'Статья'] = df['SKK2']
df.loc[df['SK']=='52', 'Статья'] = df['SKK2']

C:\Users\SMOLEN~1\AppData\Local\Temp\16/ipykernel_6000/4240791648.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Статья'] = ''
C:\Users\Smolenyshev_OV\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [228]:
df["Статья"].unique()

array(['2.6.13 Банковские услуги', '2.1.1.2\tУплата НДФЛ ',
       '6.03\tПредоставление займов', '2.3.7.1\tАренда земли',
       '1.4.9\tПрочие поступления (Прочие операционные поступления)',
       '5.09\tПоступление % по ценным бумагам и иным финансовым вложениям',
       '5.04\tВозврат депозитов ',
       '2.1.1.1\tВыплата заработной платы, отпускных и прочих выплат',
       '2.3.1.1\tСтационарная телефонная связь',
       '2.2.5.5\tРасходы на прочие ТМЦ (Эксплуатация - Инженерные сети и оборудование)',
       '2.2.5.2 Прочие ТМЦ (Эксплуатация - Здания и сооружения)',
       '2.2.4.3\t Инвентарь',
       '2.2.3.5\tИнструмент (Эксплуатация - Инженерные сети и оборудование)',
       '2.3.1.4\tИнтернет Телефония',
       '2.3.4.4\tПриродоохранные мероприятия',
       '2.2.3.2\tМерительные приборы, эталоны',
       '2.2.5.4\tПрочие ТМЦ (Эксплуатация - Транспортные средства)',
       '2.6.2.2\tСпецпитание (вода питьевая)',
       '2.3.3.3\tПрочие (Благоустройство (территории, помещений)

In [229]:
df['Статья'].str[:3].unique()

array(['2.6', '2.1', '6.0', '2.3', '1.4', '5.0', '2.2', '6.9', '1.2',
       '4.1', '2.5', '4.2', '2.4', '7.1', '7.2', '6.1', '3.1', '1.3',
       '3.2', '4.4', '4.5', 'Воз'], dtype=object)

In [230]:
#Столбец Статья2 с вгрегированными группами статей

df['Группа'] = ''

df.loc[df['Статья'].str[:1]=='1', 'Группа'] = '1. Приток от опер-ной деятельности'
df.loc[df['Статья'].str[:3]=='2.1', 'Группа'] = '2.1 Расходы на персонал' 
df.loc[df['Статья'].str[:3]=='2.2', 'Группа'] = '2.2 Материалы'
df.loc[df['Статья'].str[:3]=='2.3', 'Группа'] = '2.3 Работы и услуги'
df.loc[df['Статья'].str[:3]=='2.4', 'Группа'] = '2.4 Страхование'
df.loc[df['Статья'].str[:3]=='2.5', 'Группа'] = '2.5 Налоги, сборы и платежи'
df.loc[df['Статья'].str[:3]=='2.6', 'Группа'] = '2.6 Прочие платежи'
df.loc[df['Статья'].str[:1]=='3', 'Группа'] = '3. Приток от инвест-ной деятельности'
df.loc[df['Статья'].str[:1]=='4', 'Группа'] = '4. Отток от инвест-ной деятельности'
df.loc[df['Статья'].str[:1]=='5', 'Группа'] = '5. Приток от фин-ной деятельности'
df.loc[df['Статья'].str[:1]=='6', 'Группа'] = '6. Отток от фин-ной деятельности'
df.loc[df['Статья'].str[:1]=='7', 'Группа'] = '7. Внутренние перемещения'
df.loc[df['Статья'].str[:7]=='Возврат', 'Группа'] = '9. Возврат'

C:\Users\SMOLEN~1\AppData\Local\Temp\16/ipykernel_6000/1203533715.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Группа'] = ''
C:\Users\Smolenyshev_OV\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [231]:
df['Группа'].unique()

# df['Статья2']

array(['2.6 Прочие платежи', '2.1 Расходы на персонал',
       '6. Отток от фин-ной деятельности', '2.3 Работы и услуги',
       '1. Приток от опер-ной деятельности',
       '5. Приток от фин-ной деятельности', '2.2 Материалы',
       '4. Отток от инвест-ной деятельности',
       '2.5 Налоги, сборы и платежи', '2.4 Страхование',
       '7. Внутренние перемещения',
       '3. Приток от инвест-ной деятельности', '9. Возврат'], dtype=object)

In [232]:
#удаляю знак \t:

df['Статья'].replace(to_replace="\t", value=' ', inplace=True, limit=None, regex=True)

# df['Статья'].unique()

C:\Users\Smolenyshev_OV\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [233]:
# df.info()

In [234]:
# дату в нормальный формат
# df['date'] = pd.to_datetime(df['date'])

# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead

df['date'] = df['date'].apply(pd.to_datetime).copy()

C:\Users\SMOLEN~1\AppData\Local\Temp\16/ipykernel_6000/1687103453.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].apply(pd.to_datetime).copy()


In [235]:
# df['date']

In [236]:
# нахождение крайней даты
lastday = df['date'].sort_values(ascending=True).tail(1).astype('datetime64[ns]')

#
print('lastday: ', lastday)
print('-------')

#привоэу в более менее удобный формат для выдачи в текст заголовка
lastday2 = lastday.astype('str').values

print('lastday2: ', lastday2)

lastday:  164392   2021-10-18
Name: date, dtype: datetime64[ns]
-------
lastday2:  ['2021-10-18']


In [237]:
lastday

164392   2021-10-18
Name: date, dtype: datetime64[ns]

In [238]:
# сегодня

today = pd.to_datetime('today')

today

Timestamp('2021-10-20 11:43:22.434985')

In [239]:
# сегодня в номальный вид

today2 = dt.datetime.today().strftime("%Y-%m-%d")

today2

'2021-10-20'

In [240]:
# эксперементирую

today3 = pd.to_datetime(today2)

today3

Timestamp('2021-10-20 00:00:00')

In [241]:
# библиотека для работы с датами и днями
from pandas.tseries.offsets import MonthEnd

In [242]:
# последний день текущего месяца

dem = pd.to_datetime(today2)+MonthEnd(1)

dem

Timestamp('2021-10-31 00:00:00')

In [243]:
dem2 = dem.strftime('%Y-%m-%d')

dem2

'2021-10-31'

In [244]:
# Отбор только по периоду
# df = df[df['date']>= '2021-01-01']

# df = df[(df['date']>= '2021-01-01') & (df['date']<= '2021-08-31')]

df = df[(df['date']>= '2021-01-01') & (df['date']<= dem)]

In [245]:
# стролбцы с периодами
df['Год'] = df['date'].dt.year.astype('Int64').values
df['Месяц'] = df['date'].dt.month.astype('Int64').values
df['День'] = df['date'].dt.day.astype('Int64').values

In [246]:
#только год и месяц
df['YM'] = df['date'].dt.to_period('M')

In [247]:
# Числовое поле в тыс.
df['ФАКТ'] = pd.to_numeric(df["Summa"]/1000).round(2)




In [248]:
#Приток +, отток -С Лучше делать через .loc

# df['Ф-т'] = np.where(df['SD']=='50', df['ФАКТ'],\
#                np.where(df['SD']=='51', df['ФАКТ'],\
#                np.where(df['SD']=='52', df['ФАКТ'],\
#                np.where(df['SK']=='50', df['ФАКТ']*(-1),\
#                np.where(df['SK']=='51', df['ФАКТ']*(-1),\
#                np.where(df['SK']=='50', df['ФАКТ']*(-1), 
#                ''))))))


# перешел на .lov
df['Факт'] = ''

df.loc[df['SD']=='50', 'Ф-т'] = df['ФАКТ']
df.loc[df['SD']=='51', 'Ф-т'] = df['ФАКТ']
df.loc[df['SD']=='52', 'Ф-т'] = df['ФАКТ']
df.loc[df['SK']=='50', 'Ф-т'] = df['ФАКТ']*-1
df.loc[df['SK']=='51', 'Ф-т'] = df['ФАКТ']*-1
df.loc[df['SK']=='52', 'Ф-т'] = df['ФАКТ']*-1


In [249]:
# df['Ф-т'].unique()

In [250]:
# округление
df['Факт'] = pd.to_numeric(df['Ф-т']).round(2)

In [251]:
# df.tail()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4118 entries, 94614 to 164392
Data columns (total 39 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         4118 non-null   int64         
 1   Depatment  4118 non-null   object        
 2   Doc_id     4118 non-null   object        
 3   date       4118 non-null   datetime64[ns]
 4   SD         4118 non-null   object        
 5   PodrDt     4118 non-null   object        
 6   SKD1       4118 non-null   object        
 7   SKD1_cod   4118 non-null   object        
 8   SKD1_tip   4118 non-null   object        
 9   SKD2       4118 non-null   object        
 10  SKD2_cod   4118 non-null   object        
 11  SKD2_tip   4118 non-null   object        
 12  SKD3       4118 non-null   object        
 13  SKD3_cod   4118 non-null   object        
 14  SKD3_tip   4118 non-null   object        
 15  SK         4118 non-null   object        
 16  PodrKt     4118 non-null   object   

In [252]:
# выбираю только нужные столбцы
# df = df[['date', 'YM', 'Год','Месяц','День', 'SD', 'SK', 'Статья', 'Статья2', 'ФАКТ', 'Ф-т']]

df = df[['date', 'YM', 'Год', 'Месяц', 'День', 'SD', 'SK', 'Группа', 'Статья', 'Факт', 'SKD1', 'SKD2', 'SKD3', 'SKK1', 'SKK2', 'SKK3']]

In [253]:
df.info()
# df.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4118 entries, 94614 to 164392
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4118 non-null   datetime64[ns]
 1   YM      4118 non-null   period[M]     
 2   Год     4118 non-null   Int64         
 3   Месяц   4118 non-null   Int64         
 4   День    4118 non-null   Int64         
 5   SD      4118 non-null   object        
 6   SK      4118 non-null   object        
 7   Группа  4118 non-null   object        
 8   Статья  4118 non-null   object        
 9   Факт    4118 non-null   float64       
 10  SKD1    4118 non-null   object        
 11  SKD2    4118 non-null   object        
 12  SKD3    4118 non-null   object        
 13  SKK1    4118 non-null   object        
 14  SKK2    4118 non-null   object        
 15  SKK3    4118 non-null   object        
dtypes: Int64(3), datetime64[ns](1), float64(1), object(10), period[M](1)
memory usage: 688.0+ 

In [254]:
# df только по притоку ДС от опер и инвест деят-ти

df_pr = df[(df['Группа']=='1. Приток от опер-ной деятельности')  | (df['Группа']=='3. Приток от инвест-ной деятельности')]

df_pr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 701 entries, 94643 to 164354
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    701 non-null    datetime64[ns]
 1   YM      701 non-null    period[M]     
 2   Год     701 non-null    Int64         
 3   Месяц   701 non-null    Int64         
 4   День    701 non-null    Int64         
 5   SD      701 non-null    object        
 6   SK      701 non-null    object        
 7   Группа  701 non-null    object        
 8   Статья  701 non-null    object        
 9   Факт    701 non-null    float64       
 10  SKD1    701 non-null    object        
 11  SKD2    701 non-null    object        
 12  SKD3    701 non-null    object        
 13  SKK1    701 non-null    object        
 14  SKK2    701 non-null    object        
 15  SKK3    701 non-null    object        
dtypes: Int64(3), datetime64[ns](1), float64(1), object(10), period[M](1)
memory usage: 95.2+ KB

In [255]:
# сводная табл по ОДДС по Статьям и Группам

pt1 = pd.pivot_table(df, index = ['Группа', 'Статья'], values = ['Факт'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt1

Факт  \
Год                                                                                       2021   
Месяц                                                                                        1   
Группа                             Статья                                                        
1. Приток от опер-ной деятельности 1.2.1 Поступления от аренды помещений                         
                                   1.2.3 Поступления от прочей аренды                    33.42   
                                   1.3.3 Реализация услуг железнодорожного транспорта  1590.22   
                                   1.3.9 Реализация услуг IT службы                              
                                   1.4.1 Продажа ТМЦ                                  23840.00   
...                                                                                        ...   
6. Отток от фин-ной деятельности   6.10.1.4 Транспортные средства (Финансовая арен...            
7. Внутренние перемещения          7.1 Внутренние перемещения по расчетному счету         0.00   
9. Возврат                         Возврат денежных средств от поставщика                        
                                   Возврат оплаты клиенту                                        
All                                                                                    9065.04   

                                                                                                \
Год                                                                                              
Месяц                                                                                        2   
Группа                             Статья                                                        
1. Приток от опер-ной деятельности 1.2.1 Поступления от аренды помещений                  2.00   
                                   1.2.3 Поступления от прочей аренды                    35.18   
                                   1.3.3 Реализация услуг железнодорожного транспорта  1255.31   
                                   1.3.9 Реализация услуг IT службы                              
                                   1.4.1 Продажа ТМЦ                                  65817.01   
...                                                                                        ...   
6. Отток от фин-ной деятельности   6.10.1.4 Транспортные средства (Финансовая арен...   -36.57   
7. Внутренние перемещения          7.1 Внутренние перемещения по расчетному счету         0.00   
9. Возврат                         Возврат денежных средств от поставщика                        
                                   Возврат оплаты клиенту                                        
All                                                                                   26964.28   

                                                                                                 \
Год                                                                                               
Месяц                                                                                         3   
Группа                             Статья                                                         
1. Приток от опер-ной деятельности 1.2.1 Поступления от аренды помещений                   2.00   
                                   1.2.3 Поступления от прочей аренды                     34.18   
                                   1.3.3 Реализация услуг железнодорожного транспорта   1314.76   
                                   1.3.9 Реализация услуг IT службы                       68.50   
                                   1.4.1 Продажа ТМЦ                                   86093.33   
...                                                                                         ...   
6. Отток от фин-ной деятельности   6.10.1.4 Транспортные средства (Финансовая арен...  -1220.97   
7. Внутренние перемещения          7.1 Внутренние перемещен

In [256]:
# сводная табл по ОДДС только по группам

pt2 = pd.pivot_table(df, index = ['Группа'], values = ['Факт'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt2

Факт                                \
Год                                       2021                                 
Месяц                                        1         2         3         4   
Группа                                                                         
1. Приток от опер-ной деятельности    25463.82  67131.00  87512.77  84189.05   
2.1 Расходы на персонал              -13981.78 -18359.88 -23099.06 -25230.26   
2.2 Материалы                        -10496.98  -7670.10  -9957.80  -8021.01   
2.3 Работы и услуги                   -1216.59  -4102.68 -11813.46  -7293.61   
2.4 Страхование                                            -185.61   -320.17   
2.5 Налоги, сборы и платежи           -2152.70  -1637.33 -39037.98 -15288.22   
2.6 Прочие платежи                    -6213.31 -10287.82  -7475.40  -9346.28   
3. Приток от инвест-ной деятельности   3580.78  13288.14  21950.30  13580.50   
4. Отток от инвест-ной деятельности    -430.83  -7360.48 -12981.23 -33437.64   
5. Приток от фин-ной деятельности     17012.63             7000.00  50000.00   
6. Отток от фин-ной деятельности      -2500.00  -4036.57 -22220.97  -4611.77   
7. Внутренние перемещения                 0.00      0.00      0.00      0.00   
9. Возврат                                                   75.00             
All                                    9065.04  26964.28 -10233.44  44220.59   

                                                                              \
Год                                                                            
Месяц                                        5         6         7         8   
Группа                                                                         
1. Приток от опер-ной деятельности    54991.34  99807.97  66941.94  69698.56   
2.1 Расходы на персонал              -17242.91 -19609.19 -18166.57 -15846.63   
2.2 Материалы                         -4272.70  -4738.89  -3935.35  -4998.96   
2.3 Работы и услуги                   -5649.86 -11887.05 -10938.56 -27124.49   
2.4 Страхование                        -143.27   -282.63   -186.64    -58.44   
2.5 Налоги, сборы и платежи           -5920.17  -5995.15  -8812.03  -8935.36   
2.6 Прочие платежи                    -7873.66 -16567.82  -8999.66 -11530.45   
3. Приток от инвест-ной деятельности   7713.63  16801.95   2189.60    690.00   
4. Отток от инвест-ной деятельности  -10641.48 -10688.61 -18785.10  -6065.57   
5. Приток от фин-ной деятельности               50011.99                       
6. Отток от фин-ной деятельности      -1683.67 -70032.61  -8084.25  -7504.08   
7. Внутренние перемещения                 0.00      0.00                0.00   
9. Возврат                                        -22.18               22.82   
All                                    9277.25  26797.78  -8776.62 -11652.60   

                                                                     
Год                                                             All  
Месяц                                        9        10             
Группа                                                               
1. Приток от опер-ной деятельности    95666.35  35407.78  686810.58  
2.1 Расходы на персонал              -14586.13 -10521.75 -176644.16  
2.2 Материалы                         -4829.13  -2865.10  -61786.02  
2.3 Работы и услуги                  -12340.57 -17776.58 -110143.45  
2.4 Страхование                                            -1176.76  
2.5 Налоги, сборы и платежи           -8653.86    -33.52  -96466.32  
2.6 Прочие платежи                    -8716.11   -578.00  -87588.51  
3. Приток от инвест-ной деятельности   6412.00   2400.00   88606.90  
4. Отток от инвест-ной деятельности  -19532.64  -7923.63 -127847.21  
5. Приток от фин-ной деятельности                         124024.62  
6. Отток от фин-ной деятельности      -8651.38  -3911.78 -133237.08  
7. Внутренние перемещения                 0.00      0.00       0.00  
9. Возврат                              

In [257]:
# сводная табл по притоку ДС по опер и иныест деят по Статьям, Группам и контрагекнтам

pt3 = pd.pivot_table(df_pr, index = ['Группа', 'Статья', 'SKK1'], values = ['Факт'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt3

Факт  \
Год                                                                                                                        2021   
Месяц                                                                                                                         1   
Группа                               Статья                                             SKK1                                      
1. Приток от опер-ной деятельности   1.2.1 Поступления от аренды помещений              Алифанов Роман Игоревич                   
                                                                                        ВЫМПЕЛКОМ ПАО                             
                                                                                        ПРОМОСФЕРА ООО                            
                                                                                        ТТК-СВЯЗЬ ООО                             
                                     1.2.3 Поступления от прочей аренды                 ВЫМПЕЛКОМ ПАО                     33.42   
...                                                                                                                         ...   
3. Приток от инвест-ной деятельности 3.1.3 Транспортные средства (Выручка от продажи... Филлипенков Антон Андреевич ЧЛ            
                                                                                        ЦЕНТРУС 1520 ООО                          
                                                                                        Шикерханов Гаджибала Рафикович            
                                     3.1.4 Инженерные сети и оборудование (Выручка о... РЕГИОН ПЛЮС ООО                           
All                                                                                                                    29044.60   

                                                                                                                                 \
Год                                                                                                                               
Месяц                                                                                                                         2   
Группа                               Статья                                             SKK1                                      
1. Приток от опер-ной деятельности   1.2.1 Поступления от аренды помещений              Алифанов Роман Игоревич            2.00   
                                                                                        ВЫМПЕЛКОМ ПАО                             
                                                                                        ПРОМОСФЕРА ООО                            
                                                                                        ТТК-СВЯЗЬ ООО                             
                                     1.2.3 Поступления от прочей аренды                 ВЫМПЕЛКОМ ПАО                     34.18   
...                                                                                                                         ...   
3. Приток от инвест-ной деятельности 3.1.3 Транспортные средства (Выручка от продажи... Филлипенков Антон Андреевич ЧЛ            
                                                                                        ЦЕНТРУС 1520 ООО                          
                                                                                        Шикерханов Гаджибала Рафикович            
                                     3.1.4 Инженерные сети и оборудование (Выручка о... РЕГИОН ПЛЮС ООО                           
All                                                                                                                    80419.14   

                                                                                                                                  \
Год                                                        

In [258]:
# pt_f1 = pd.pivot_table(df, index = ['Статья'], values = ['ФАКТ'], aggfunc=np.sum, fill_value='', margins=False)

pt_f1 = pd.pivot_table(df, index = ['Группа'], values = ['Факт'], aggfunc=np.sum, fill_value='', margins=False)

pt_f1

,Факт
Группа,
1. Приток от опер-ной деятельности,686810.58
2.1 Расходы на персонал,-176644.16
2.2 Материалы,-61786.02
2.3 Работы и услуги,-110143.45
2.4 Страхование,-1176.76
"2.5 Налоги, сборы и платежи",-96466.32
2.6 Прочие платежи,-87588.51
3. Приток от инвест-ной деятельности,88606.90
4. Отток от инвест-ной деятельности,-127847.21


In [259]:
pt_f2 = pd.pivot_table(df, index = ['Группа'], values = ['Факт'], columns='YM', aggfunc=np.sum, fill_value='', margins=False)

pt_f2

Факт                                \
YM                                     2021-01   2021-02   2021-03   2021-04   
Группа                                                                         
1. Приток от опер-ной деятельности    25463.82  67131.00  87512.77  84189.05   
2.1 Расходы на персонал              -13981.78 -18359.88 -23099.06 -25230.26   
2.2 Материалы                        -10496.98  -7670.10  -9957.80  -8021.01   
2.3 Работы и услуги                   -1216.59  -4102.68 -11813.46  -7293.61   
2.4 Страхование                                            -185.61   -320.17   
2.5 Налоги, сборы и платежи           -2152.70  -1637.33 -39037.98 -15288.22   
2.6 Прочие платежи                    -6213.31 -10287.82  -7475.40  -9346.28   
3. Приток от инвест-ной деятельности   3580.78  13288.14  21950.30  13580.50   
4. Отток от инвест-ной деятельности    -430.83  -7360.48 -12981.23 -33437.64   
5. Приток от фин-ной деятельности     17012.63             7000.00  50000.00   
6. Отток от фин-ной деятельности      -2500.00  -4036.57 -22220.97  -4611.77   
7. Внутренние перемещения                 0.00      0.00      0.00      0.00   
9. Возврат                                                   75.00             

                                                                              \
YM                                     2021-05   2021-06   2021-07   2021-08   
Группа                                                                         
1. Приток от опер-ной деятельности    54991.34  99807.97  66941.94  69698.56   
2.1 Расходы на персонал              -17242.91 -19609.19 -18166.57 -15846.63   
2.2 Материалы                         -4272.70  -4738.89  -3935.35  -4998.96   
2.3 Работы и услуги                   -5649.86 -11887.05 -10938.56 -27124.49   
2.4 Страхование                        -143.27   -282.63   -186.64    -58.44   
2.5 Налоги, сборы и платежи           -5920.17  -5995.15  -8812.03  -8935.36   
2.6 Прочие платежи                    -7873.66 -16567.82  -8999.66 -11530.45   
3. Приток от инвест-ной деятельности   7713.63  16801.95   2189.60    690.00   
4. Отток от инвест-ной деятельности  -10641.48 -10688.61 -18785.10  -6065.57   
5. Приток от фин-ной деятельности               50011.99                       
6. Отток от фин-ной деятельности      -1683.67 -70032.61  -8084.25  -7504.08   
7. Внутренние перемещения                 0.00      0.00                0.00   
9. Возврат                                        -22.18               22.82   

                                                          
YM                                     2021-09   2021-10  
Группа                                                    
1. Приток от опер-ной деятельности    95666.35  35407.78  
2.1 Расходы на персонал              -14586.13 -10521.75  
2.2 Материалы                         -4829.13  -2865.10  
2.3 Работы и услуги                  -12340.57 -17776.58  
2.4 Страхование                                           
2.5 Налоги, сборы и платежи           -8653.86    -33.52  
2.6 Прочие платежи                    -8716.11   -578.00  
3. Приток от инвест-ной деятельности   6412.00   2400.00  
4. Отток от инвест-ной деятельности  -19532.64  -7923.63  
5. Приток от фин-ной деятельности                         
6. Отток от фин-ной деятельности      -8651.38  -3911.78  
7. Внутренние перемещения                 0.00      0.00  
9. Возврат                                7.31

In [260]:
#Сегодня
now = datetime.today()

#now.month
#now.day
#now.hour
#now.minute

# текущий день
td = now.day

# текущий месяц
tm = now.month

tm

10

In [261]:
#Делаю табл только за текущий месяц

dfm = df[df["Месяц"] == tm] # tm


In [262]:
#Делаю табл притоку ДС по опер и инвест деят-ти только за текущий месяц

df_prm = df_pr[df_pr["Месяц"] == tm] # tm

In [263]:
# сводная табл по ОДДС по Статьям и Группам за текущий месяц

pt4 = pd.pivot_table(dfm, index = ['Группа', 'Статья'], values = ['Факт'], columns = ['Год', 'Месяц', 'День'], aggfunc=np.sum, fill_value='', margins=True)

pt4

Факт  \
Год                                                                                         2021   
Месяц                                                                                         10   
День                                                                                           1   
Группа                               Статья                                                        
1. Приток от опер-ной деятельности   1.2.1 Поступления от аренды помещений                         
                                     1.2.3 Поступления от прочей аренды                            
                                     1.3.3 Реализация услуг железнодорожного транспорта            
                                     1.4.1 Продажа ТМЦ                                   2220.11   
                                     1.4.10 Возврат подотчетных средств                            
                                     1.4.9 Прочие поступления (Прочие операционные п...            
2.1 Расходы на персонал              2.1.1.1 Выплата заработной платы, отпускных и п...   -33.28   
                                     2.1.1.2 Уплата НДФЛ                                   -4.96   
                                     2.1.1.3.1. Страховые взносы в ФСС                             
                                     2.1.1.3.2. Страховые взносы в ФОМС                            
                                     2.1.1.3.3. Страховые взносы в ПФ                              
                                     2.1.1.3.4. Страховые взносы от НС                             
                                     2.1.2.1 Обязательное (Обучение и аттестация пер...            
                                     2.1.3.2 Расходы на съем жилья                                 
2.2 Материалы                        2.2.2.2 Электроэнергия                                        
                                     2.2.2.5 Расходы на ГСМ (Эксплуатация - Транспор...            
                                     2.2.4.4 Канцтовары, хозтовары, АХО                            
                                     2.2.5.4 Прочие ТМЦ (Эксплуатация - Транспортные...            
                                     2.2.5.5 Расходы на прочие ТМЦ (Эксплуатация - И...            
                                     2.2.5.6 Прочие ТМЦ (Эксплуатация - Телекоммуник...  -136.78   
2.3 Работы и услуги                  2.3.1.1 Стационарная телефонная связь                         
                                     2.3.1.2 Мобильная связь                                       
                                     2.3.1.3 Почтовые  расходы                                     
                                     2.3.1.4 Интернет Телефония                                    
                                     2.3.3.1 Вода и стоки                                          
                                     2.3.3.1 ТБО, Стоки                                    -5.45   
                                     2.3.4.4 Природоохранные мероприятия                -1263.93   
                                     2.3.4.6 Сертификация и лицензирование бизнеса                 
                                     2.3.4.7 Проектные работы, обследование зданий                 
                                     2.3.5.1 Эксплуатация и ремонт зданий и сооружения)            
                                     2.3.5.3 Услуги сторонних организаций по эксплуа...   -27.12   
                                     2.3.6.1 Услуги сторонних организаций (Эксплуата...            
                                     2.3.7.1 Аренда земли                                          
                                     2.3.7.2 Аренда зданий, сооружений                    -36.00   
                                     2.3.9.3 Прочие (Расходы на логистику)                -30.00   
2.5 Налоги, сборы и платежи          2.5.6 Пени и штрафы                                   -

In [264]:
# сводная табл по притоку ДС по опер и иныест деят по Статьям, Группам и контрагекнтам

pt5 = pd.pivot_table(df_prm, index = ['Группа', 'Статья', 'SKK1'], values = ['Факт'], columns = ['Год', 'Месяц', 'День'], aggfunc=np.sum, fill_value='', margins=True)

pt5

Факт  \
Год                                                                                                                                  2021   
Месяц                                                                                                                                  10   
День                                                                                                                                    1   
Группа                               Статья                                             SKK1                                                
1. Приток от опер-ной деятельности   1.2.1 Поступления от аренды помещений              ВЫМПЕЛКОМ ПАО                                       
                                                                                        ТТК-СВЯЗЬ ООО                                       
                                     1.2.3 Поступления от прочей аренды                 ВЫМПЕЛКОМ ПАО                                       
                                     1.3.3 Реализация услуг железнодорожного транспорта НЕФТЕТРАНССЕРВИС АО                                 
                                                                                        РЕД ХОЛТ ООО                                        
                                                                                        ТРАНС-ЭКСПО ООО                                     
                                     1.4.1 Продажа ТМЦ                                  Будсолович Руслан Николаевич                        
                                                                                        ВТОРМЕТАЛЛ ООО                            1000.00   
                                                                                        Клям Андрей Андреевич                       20.11   
                                                                                        ПРОМОСФЕРА ООО                            1200.00   
                                                                                        ПРОТОН ООО                                          
                                                                                        РЕГИОН ПЛЮС ООО                                     
                                     1.4.10 Возврат подотчетных средств                 Шаповалова Татьяна Николаевна                       
                                     1.4.9 Прочие поступления (Прочие операционные п... 91.1.04   Прочие внереализационные доходы           
                                                                                        Пятницкий Яков Сергеевич                            
3. Приток от инвест-ной деятельности 3.1.4 Инженерные сети и оборудование (Выручка о... РЕГИОН ПЛЮС ООО                                     
All                                                                                                                               2220.11   

                                                                                                                                           \
Год                                                                                                                                         
Месяц                                                                                                                                       
День                                                                                                                                    4   
Группа                               Статья                                             SKK1                                                
1. Приток от опер-ной деятельности   1.2.1 Поступления от аренды помещений              ВЫМПЕЛКОМ ПАО                                       
                                                                                        ТТК-СВЯЗЬ ООО                                       
                                     1.2.3 

In [265]:
# загрузка бюджета
# df_b = pd.read_excel('X:\\ФинДир\\Base\\pt_bdds21.xlsx', sheet_name='base')

#полный путь
# df_b = pd.read_excel('\\\\VPROMFS01\\Share\\ФинДир\\Base\\pt_bdds21_v2.xlsx', sheet_name='base')

# \\172.16.50.6\Smolenyshev_OV\Уровень_1\Base

df_b = pd.read_excel('\\\\172.16.50.6\\Smolenyshev_OV\\Уровень_1\\Base\\pt_bdds21_v2.xlsx', sheet_name='base')

In [266]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Код        730 non-null    float64       
 1   Раздел ДС  754 non-null    object        
 2   Статья     754 non-null    object        
 3   Период     754 non-null    datetime64[ns]
 4   Значение   754 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 29.6+ KB


In [267]:
# Отбор только по периоду

# df_b = df_b[(df_b['Период']>= '2021-01-01') & (df_b['Период']<= '2021-08-31')]

df_b = df_b[(df_b['Период']>= '2021-01-01') & (df_b['Период']<= dem)]

In [268]:
df_b.tail()

,Код,Раздел ДС,Статья,Период,Значение
747,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-06-01,27310.30
748,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-07-01,22891.38
749,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-08-01,17475.31
750,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-09-01,17688.00
751,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-10-01,18665.69


In [269]:
#  новый столбец с названием статей. В бюджете отток по инвест деят без цифр кода в начале строки, поэтому беру из столбца Код

df_b['Группа'] = ''

df_b.loc[df_b['Статья'].str[:1]=='1', 'Группа'] = '1. Приток от опер-ной деятельности'
df_b.loc[df_b['Статья'].str[:3]=='2.1', 'Группа'] = '2.1 Расходы на персонал' 
df_b.loc[df_b['Статья'].str[:3]=='2.2', 'Группа'] = '2.2 Материалы'
df_b.loc[df_b['Статья'].str[:3]=='2.3', 'Группа'] = '2.3 Работы и услуги'
df_b.loc[df_b['Статья'].str[:3]=='2.4', 'Группа'] = '2.4 Страхование'
df_b.loc[df_b['Статья'].str[:3]=='2.5', 'Группа'] = '2.5 Налоги, сборы и платежи'
df_b.loc[df_b['Статья'].str[:3]=='2.6', 'Группа'] = '2.6 Прочие платежи'
df_b.loc[df_b['Статья'].str[:1]=='3', 'Группа'] = '3. Приток от инвест-ной деятельности'
df_b.loc[df_b['Статья'].str[:1]=='4', 'Группа'] = '4. Отток от инвест-ной деятельности'
df_b.loc[df_b['Статья']=='Метанол', 'Группа'] = '4. Отток от инвест-ной деятельности'
df_b.loc[df_b['Статья']=='ОЭС', 'Группа'] = '4. Отток от инвест-ной деятельности'
df_b.loc[df_b['Статья']=='Инфраструткрура площадки', 'Группа'] = '4. Отток от инвест-ной деятельности'
df_b.loc[df_b['Статья']=='Выкуп земли', 'Группа'] = '4. Отток от инвест-ной деятельности'
df_b.loc[df_b['Статья'].str[:1]=='5', 'Группа'] = '5. Приток от фин-ной деятельности'
df_b.loc[df_b['Статья'].str[:1]=='6', 'Группа'] = '6. Отток от фин-ной деятельности'
df_b.loc[df_b['Статья'].str[:1]=='7', 'Группа'] = '7. Внутренние перемещения'
df_b.loc[df_b['Статья'].str[:7]=='Возврат', 'Группа'] = '9. Возврат'

In [270]:
# преобразование в число
df_b['БЮДЖЕТ'] = pd.to_numeric(df_b['Значение']).round(2)

In [271]:
df_b['Группа'].str[:1].unique()

array(['', '1', '2', '3', '4', '5', '6'], dtype=object)

In [272]:
# Create new column using conditional indexing
df_b["План"] = ""
df_b.loc[df_b['Группа'].str[:1] == '1', "План"] = df_b['БЮДЖЕТ']
df_b.loc[df_b['Группа'].str[:1] == '3', "План"] = df_b['БЮДЖЕТ']
df_b.loc[df_b['Группа'].str[:1] == '5', "План"] = df_b['БЮДЖЕТ']
df_b.loc[df_b['Группа'].str[:1] == '2', "План"] = df_b['БЮДЖЕТ']*-1
df_b.loc[df_b['Группа'].str[:1] == '4', "План"] = df_b['БЮДЖЕТ']*-1
df_b.loc[df_b['Группа'].str[:1] == '6', "План"] = df_b['БЮДЖЕТ']*-1
df_b

,Код,Раздел ДС,Статья,Период,Значение,Группа,БЮДЖЕТ,План
0,0.00,Остаток ден. средств на нач пер,Остаток ден. средств на нач пер,2021-01-01,1804.38,,1804.38,
1,0.00,Остаток ден. средств на нач пер,Остаток ден. средств на нач пер,2021-02-01,10871.48,,10871.48,
2,0.00,Остаток ден. средств на нач пер,Остаток ден. средств на нач пер,2021-03-01,37786.48,,37786.48,
3,0.00,Остаток ден. средств на нач пер,Остаток ден. средств на нач пер,2021-04-01,3081.19,,3081.19,
4,0.00,Остаток ден. средств на нач пер,Остаток ден. средств на нач пер,2021-05-01,4770.64,,4770.64,
...,...,...,...,...,...,...,...,...
747,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-06-01,27310.30,,27310.30,
748,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-07-01,22891.38,,22891.38,
749,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-08-01,17475.31,,17475.31,
750,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-09-01,17688.00,,17688.00,


In [273]:
df_b['План'] = pd.to_numeric(df_b['План']).round(2)

In [274]:
# переименовал столбец
df_b = df_b.rename(columns= {'Период': 'date'})

# df_b = df_b.rename(columns= {'Период': 'date', 'Значение': 'ST'})

In [275]:
# стролбцы с периодами
df_b['Год'] = df_b['date'].dt.year.astype('Int64').values
df_b['Месяц'] = df_b['date'].dt.month.astype('Int64').values
df_b['День'] = df_b['date'].dt.day.astype('Int64').values

In [276]:
#только год и месяц
df_b['YM'] = df_b['date'].dt.to_period('M')

In [277]:
# df_b.tail()
# df_b.head()
df_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 635 entries, 0 to 751
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Код        615 non-null    float64       
 1   Раздел ДС  635 non-null    object        
 2   Статья     635 non-null    object        
 3   date       635 non-null    datetime64[ns]
 4   Значение   635 non-null    float64       
 5   Группа     635 non-null    object        
 6   БЮДЖЕТ     635 non-null    float64       
 7   План       483 non-null    float64       
 8   Год        635 non-null    Int64         
 9   Месяц      635 non-null    Int64         
 10  День       635 non-null    Int64         
 11  YM         635 non-null    period[M]     
dtypes: Int64(3), datetime64[ns](1), float64(4), object(3), period[M](1)
memory usage: 82.5+ KB


In [278]:
# df_b['Статья'].unique()

In [279]:
# тестовая сводная табл по БДДС

pt_b0 = pd.pivot_table(df_b, index = ['Группа'], values = ['БЮДЖЕТ'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt_b0

БЮДЖЕТ                                \
Год                                       2021                                 
Месяц                                        1         2         3         4   
Группа                                                                         
                                     197105.06 370333.96 501811.04 420640.91   
1. Приток от опер-ной деятельности    25524.60  79054.00  97635.84  76879.27   
2.1 Расходы на персонал               13981.70  18414.00  20894.24  21538.80   
2.2 Материалы                         10497.05   7670.00   6708.76   5214.59   
2.3 Работы и услуги                    1216.55   4103.00   8187.90  11802.53   
2.4 Страхование                                             850.00      0.00   
2.5 Налоги, сборы и платежи            2152.50   1621.00  31227.00  14090.00   
2.6 Прочие платежи                     6211.10  10298.00   8868.00   9368.00   
3. Приток от инвест-ной деятельности   3519.80   1365.00  17600.00  26318.00   
4. Отток от инвест-ной деятельности     431.30   7361.00  52980.24  37042.38   
5. Приток от фин-ной деятельности     17012.90                0.00      0.00   
6. Отток от фин-ной деятельности       2500.00   4037.00  20225.00   2451.51   
All                                  280152.56 504256.96 766988.02 625345.99   

                                                                              \
Год                                                                            
Месяц                                        5         6         7         8   
Группа                                                                         
                                     496340.75 468968.74 383163.55 352448.56   
1. Приток от опер-ной деятельности    70685.47  49782.55  59740.47  48820.47   
2.1 Расходы на персонал               19640.43  18550.74  17917.05  18008.40   
2.2 Материалы                          4944.79   4645.30   4579.18   4983.39   
2.3 Работы и услуги                   18148.15  20488.15  18064.30  15365.10   
2.4 Страхование                           0.00     50.00      0.00      0.00   
2.5 Налоги, сборы и платежи            7700.00   5500.00   5840.00   5450.00   
2.6 Прочие платежи                    10628.00  10628.00  16038.00  10678.00   
3. Приток от инвест-ной деятельности   5675.00   4100.00   3500.00   3200.00   
4. Отток от инвест-ной деятельности   30218.14  41486.00  23585.00  23540.00   
5. Приток от фин-ной деятельности     40000.00  50000.00  20000.00  26000.00   
6. Отток от фин-ной деятельности       3723.35   1352.29   1635.85   5411.66   
All                                  707704.08 675551.77 554063.40 513905.58   

                                                                     
Год                                                             All  
Месяц                                        9        10             
Группа                                                               
                                     386338.31 293354.61 3870505.49  
1. Приток от опер-ной деятельности    39693.75  31750.23  579566.65  
2.1 Расходы на персонал               18008.40  18133.68  185087.44  
2.2 Материалы                          5559.64   6348.27   61150.97  
2.3 Работы и услуги                   11226.60   4330.35  112932.63  
2.4 Страхование                           0.00      0.00     900.00  
2.5 Налоги, сборы и платежи            3950.00   4990.00   82520.50  
2.6 Прочие платежи                    10678.00   9368.00  102763.10  
3. Приток от инвест-ной деятельности   3100.00   2500.00   70877.80  
4. Отток от инвест-ной деятельности   32454.00  17115.00  266213.06  
5. Приток от фин-ной деятельности     45000.00  30000.00  228012.90  
6. Отток от фин-ной деятельности       5704.44   2987.23   50028.33  
All                                  561713.14 420877.37 5610558.87

In [280]:
# pt_b1 = pd.pivot_table(df_b, index = ['Статья'], values = ['БЮДЖЕТ'], aggfunc=np.sum, fill_value='', margins=False)

pt_b1 = pd.pivot_table(df_b, index = ['Группа'], values = ['План'], aggfunc=np.sum, fill_value='', margins=False)

pt_b1

,План
Группа,
,0.00
1. Приток от опер-ной деятельности,579566.65
2.1 Расходы на персонал,-185087.44
2.2 Материалы,-61150.97
2.3 Работы и услуги,-112932.63
2.4 Страхование,-900.00
"2.5 Налоги, сборы и платежи",-82520.50
2.6 Прочие платежи,-102763.10
3. Приток от инвест-ной деятельности,70877.80


In [281]:
pt_b2 = pd.pivot_table(df_b, index = ['Группа'], values = ['План'], columns = ['YM'], aggfunc=np.sum, fill_value='', margins=False)

pt_b2

План                                \
YM                                     2021-01   2021-02   2021-03   2021-04   
Группа                                                                         
                                          0.00      0.00      0.00      0.00   
1. Приток от опер-ной деятельности    25524.60  79054.00  97635.84  76879.27   
2.1 Расходы на персонал              -13981.70 -18414.00 -20894.24 -21538.80   
2.2 Материалы                        -10497.05  -7670.00  -6708.76  -5214.59   
2.3 Работы и услуги                   -1216.55  -4103.00  -8187.90 -11802.53   
2.4 Страхование                                            -850.00      0.00   
2.5 Налоги, сборы и платежи           -2152.50  -1621.00 -31227.00 -14090.00   
2.6 Прочие платежи                    -6211.10 -10298.00  -8868.00  -9368.00   
3. Приток от инвест-ной деятельности   3519.80   1365.00  17600.00  26318.00   
4. Отток от инвест-ной деятельности    -431.30  -7361.00 -52980.24 -37042.38   
5. Приток от фин-ной деятельности     17012.90                0.00      0.00   
6. Отток от фин-ной деятельности      -2500.00  -4037.00 -20225.00  -2451.51   

                                                                              \
YM                                     2021-05   2021-06   2021-07   2021-08   
Группа                                                                         
                                          0.00      0.00      0.00      0.00   
1. Приток от опер-ной деятельности    70685.47  49782.55  59740.47  48820.47   
2.1 Расходы на персонал              -19640.43 -18550.74 -17917.05 -18008.40   
2.2 Материалы                         -4944.79  -4645.30  -4579.18  -4983.39   
2.3 Работы и услуги                  -18148.15 -20488.15 -18064.30 -15365.10   
2.4 Страхование                           0.00    -50.00      0.00      0.00   
2.5 Налоги, сборы и платежи           -7700.00  -5500.00  -5840.00  -5450.00   
2.6 Прочие платежи                   -10628.00 -10628.00 -16038.00 -10678.00   
3. Приток от инвест-ной деятельности   5675.00   4100.00   3500.00   3200.00   
4. Отток от инвест-ной деятельности  -30218.14 -41486.00 -23585.00 -23540.00   
5. Приток от фин-ной деятельности     40000.00  50000.00  20000.00  26000.00   
6. Отток от фин-ной деятельности      -3723.35  -1352.29  -1635.85  -5411.66   

                                                          
YM                                     2021-09   2021-10  
Группа                                                    
                                          0.00      0.00  
1. Приток от опер-ной деятельности    39693.75  31750.23  
2.1 Расходы на персонал              -18008.40 -18133.68  
2.2 Материалы                         -5559.64  -6348.27  
2.3 Работы и услуги                  -11226.60  -4330.35  
2.4 Страхование                           0.00      0.00  
2.5 Налоги, сборы и платежи           -3950.00  -4990.00  
2.6 Прочие платежи                   -10678.00  -9368.00  
3. Приток от инвест-ной деятельности   3100.00   2500.00  
4. Отток от инвест-ной деятельности  -32454.00 -17115.00  
5. Приток от фин-ной деятельности     45000.00  30000.00  
6. Отток от фин-ной деятельности      -5704.44  -2987.23

In [282]:
pt_f1 = pt_f1.reset_index()
pt_b1 = pt_b1.reset_index()

In [283]:
pt_f1
# pt_b1

,Группа,Факт
0,1. Приток от опер-ной деятельности,686810.58
1,2.1 Расходы на персонал,-176644.16
2,2.2 Материалы,-61786.02
3,2.3 Работы и услуги,-110143.45
4,2.4 Страхование,-1176.76
5,"2.5 Налоги, сборы и платежи",-96466.32
6,2.6 Прочие платежи,-87588.51
7,3. Приток от инвест-ной деятельности,88606.90
8,4. Отток от инвест-ной деятельности,-127847.21
9,5. Приток от фин-ной деятельности,124024.62


In [284]:
m1 = pt_b1.merge(pt_f1, how = 'outer')
m1  # это уже на много лучше. Неявноя слияние по всем общим столбцам

# m1.reset_index()

m1

,Группа,План,Факт
0,,0.00,NaN
1,1. Приток от опер-ной деятельности,579566.65,686810.58
2,2.1 Расходы на персонал,-185087.44,-176644.16
3,2.2 Материалы,-61150.97,-61786.02
4,2.3 Работы и услуги,-112932.63,-110143.45
5,2.4 Страхование,-900.00,-1176.76
6,"2.5 Налоги, сборы и платежи",-82520.50,-96466.32
7,2.6 Прочие платежи,-102763.10,-87588.51
8,3. Приток от инвест-ной деятельности,70877.80,88606.90
9,4. Отток от инвест-ной деятельности,-266213.06,-127847.21


In [285]:
m1 = m1.set_index('Группа')

m1

,План,Факт
Группа,,
,0.00,NaN
1. Приток от опер-ной деятельности,579566.65,686810.58
2.1 Расходы на персонал,-185087.44,-176644.16
2.2 Материалы,-61150.97,-61786.02
2.3 Работы и услуги,-112932.63,-110143.45
2.4 Страхование,-900.00,-1176.76
"2.5 Налоги, сборы и платежи",-82520.50,-96466.32
2.6 Прочие платежи,-102763.10,-87588.51
3. Приток от инвест-ной деятельности,70877.80,88606.90


In [286]:
# добавляю столбцы
m1['абс.откл.'] = m1['Факт'] - m1['План']
m1['отн.откл.%'] = ((m1['абс.откл.'] / m1['План'])*100).round(2)

In [287]:
m1

,План,Факт,абс.откл.,отн.откл.%
Группа,,,,
,0.00,NaN,NaN,NaN
1. Приток от опер-ной деятельности,579566.65,686810.58,107243.93,18.50
2.1 Расходы на персонал,-185087.44,-176644.16,8443.28,-4.56
2.2 Материалы,-61150.97,-61786.02,-635.05,1.04
2.3 Работы и услуги,-112932.63,-110143.45,2789.18,-2.47
2.4 Страхование,-900.00,-1176.76,-276.76,30.75
"2.5 Налоги, сборы и платежи",-82520.50,-96466.32,-13945.82,16.90
2.6 Прочие платежи,-102763.10,-87588.51,15174.59,-14.77
3. Приток от инвест-ной деятельности,70877.80,88606.90,17729.10,25.01


In [288]:
# удаляю и обрабатываю NaN

# m2 = m1.dropna(how = 'all') # не подходит 

# m2.reset_index()
m2 = m1.fillna(0)

m2

,План,Факт,абс.откл.,отн.откл.%
Группа,,,,
,0.00,0.00,0.00,0.00
1. Приток от опер-ной деятельности,579566.65,686810.58,107243.93,18.50
2.1 Расходы на персонал,-185087.44,-176644.16,8443.28,-4.56
2.2 Материалы,-61150.97,-61786.02,-635.05,1.04
2.3 Работы и услуги,-112932.63,-110143.45,2789.18,-2.47
2.4 Страхование,-900.00,-1176.76,-276.76,30.75
"2.5 Налоги, сборы и платежи",-82520.50,-96466.32,-13945.82,16.90
2.6 Прочие платежи,-102763.10,-87588.51,15174.59,-14.77
3. Приток от инвест-ной деятельности,70877.80,88606.90,17729.10,25.01


In [289]:
# переменные с итогами
plan = m1['План'].sum().round(1)
fact = m1['Факт'].sum().round(1)
abs = (fact - plan).round(1)
otn = ((abs/plan)*100).round(2)

print(plan, fact, abs, otn)

16861.3 104635.5 87774.2 520.57


In [290]:
m0 = pt_b1.merge(pt_f1, how = 'outer')

m0 = m0.fillna(0)

In [291]:
m0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Группа  14 non-null     object 
 1   План    14 non-null     float64
 2   Факт    14 non-null     float64
dtypes: float64(2), object(1)
memory usage: 448.0+ bytes


In [292]:
# текущий месяц
tm = today.month
ty = today.year

print(tm)
print(ty)


10
2021


In [293]:
df['Группа'].unique()

array(['2.6 Прочие платежи', '5. Приток от фин-ной деятельности',
       '3. Приток от инвест-ной деятельности',
       '1. Приток от опер-ной деятельности', '2.1 Расходы на персонал',
       '2.2 Материалы', '2.3 Работы и услуги',
       '7. Внутренние перемещения', '2.5 Налоги, сборы и платежи',
       '4. Отток от инвест-ной деятельности',
       '6. Отток от фин-ной деятельности', '2.4 Страхование',
       '9. Возврат'], dtype=object)

In [294]:
# приток только тек месяц опет деят
df1 = df[(df["Месяц"] == tm) & (df['Группа'].str[:1]=='1')]

df1.tail()

,date,YM,Год,Месяц,День,SD,SK,Группа,Статья,Факт,SKD1,SKD2,SKD3,SKK1,SKK2,SKK3
164245,2021-10-15,2021-10,2021,10,15,51,62.02,1. Приток от опер-ной деятельности,1.4.1 Продажа ТМЦ,650.00,"40702810800020001286, ПАО СБЕРБАНК ООО ""ПРОМТЕХ""",1.4.1\tПродажа ТМЦ,,Будсолович Руслан Николаевич,Реализация трансформаторов № 17/337 от 30.09.2021,
164246,2021-10-15,2021-10,2021,10,15,51,62.02,1. Приток от опер-ной деятельности,1.4.1 Продажа ТМЦ,3000.00,"40702810800020001286, ПАО СБЕРБАНК ООО ""ПРОМТЕХ""",1.4.1\tПродажа ТМЦ,,ВТОРМЕТАЛЛ ООО,Поставка металлолома № 17/201 от 20.04.2021,
164351,2021-10-18,2021-10,2021,10,18,51,62.01,1. Приток от опер-ной деятельности,1.2.3 Поступления от прочей аренды,3.60,"40702810800020001286, ПАО СБЕРБАНК ООО ""ПРОМТЕХ""",1.2.3\tПоступления от прочей аренды,,ВЫМПЕЛКОМ ПАО,"Размещение ВОК V1555, д.с. № V1555 (06/8) от 1...",
164352,2021-10-18,2021-10,2021,10,18,51,62.02,1. Приток от опер-ной деятельности,1.4.1 Продажа ТМЦ,1500.00,"40702810800020001286, ПАО СБЕРБАНК ООО ""ПРОМТЕХ""",1.4.1\tПродажа ТМЦ,,ВТОРМЕТАЛЛ ООО,Поставка металлолома № 17/201 от 20.04.2021,
164354,2021-10-18,2021-10,2021,10,18,51,71.01,1. Приток от опер-ной деятельности,1.4.9 Прочие поступления (Прочие операционные ...,2.10,"40702810314100924980 ПАО АКБ ""АВАНГАРД"", ООО ""...",1.4.9\tПрочие поступления (Прочие операционные...,,Пятницкий Яков Сергеевич,,


In [295]:
# приток по опер деят только за тек мес
pritok_tm = df1['Факт'].sum().round(1)

pritok_tm

35407.8

In [296]:
# отток только тек месяц опет деят
df2 = df[(df["Месяц"] == tm) & (df['Группа'].str[:1]=='2')]

df2.tail()

,date,YM,Год,Месяц,День,SD,SK,Группа,Статья,Факт,SKD1,SKD2,SKD3,SKK1,SKK2,SKK3
164388,2021-10-18,2021-10,2021,10,18,71.01,51,2.6 Прочие платежи,2.6.17 Выдача подотчетных сумм (ПРАВИЛЬНО ИСПО...,-7.65,Пятницкий Яков Сергеевич,,,"40702810314100924980 ПАО АКБ ""АВАНГАРД"", ООО ""...",2.6.17\tВыдача подотчетных сумм (ПРАВИЛЬНО ИСП...,
164389,2021-10-18,2021-10,2021,10,18,71.01,51,2.6 Прочие платежи,2.6.17 Выдача подотчетных сумм (ПРАВИЛЬНО ИСПО...,-7.80,Иванова Софья Александровна,,,"40702810314100924980 ПАО АКБ ""АВАНГАРД"", ООО ""...",2.6.17\tВыдача подотчетных сумм (ПРАВИЛЬНО ИСП...,
164390,2021-10-18,2021-10,2021,10,18,71.01,51,2.6 Прочие платежи,2.6.17 Выдача подотчетных сумм (ПРАВИЛЬНО ИСПО...,-11.37,Иванова Софья Александровна,,,"40702810314100924980 ПАО АКБ ""АВАНГАРД"", ООО ""...",2.6.17\tВыдача подотчетных сумм (ПРАВИЛЬНО ИСП...,
164391,2021-10-18,2021-10,2021,10,18,71.01,51,2.6 Прочие платежи,2.6.05 Командировочные расходы,-14.78,Идрисов Акрам Алимович,,,"40702810314100924980 ПАО АКБ ""АВАНГАРД"", ООО ""...",2.6.05\tКомандировочные расходы,
164392,2021-10-18,2021-10,2021,10,18,71.01,51,2.6 Прочие платежи,2.6.05 Командировочные расходы,-17.91,Идрисов Акрам Алимович,,,"40702810314100924980 ПАО АКБ ""АВАНГАРД"", ООО ""...",2.6.05\tКомандировочные расходы,


In [297]:
# приток по опер деят только за тек мес
ottok_tm = df2['Факт'].sum().round(1)

ottok_tm

-31775.0

In [298]:
# График добавляю

# m0.plot(kind='barh')

# m0.plot(kind='barh', figsize=(10, 12))

# m0.plot(kind='barh', figsize=(10, 12), legend=False, grid=True)

# x = m0['План']
# x = m0[['План', 'Факт']]
# y = m0['Группа']

# fig, ax = plt.subplots(figsize=(12, 5))

# ax.plot(x, y)


# ax.set_title('Промтех. ОДДС план - факт')
# ax.legend(loc='upper left')
# ax.set_xlabel('тыс.руб.')
# ax.set_ylabel('Статьи ДДС')
# fig.tight_layout()


# plt.show()

In [299]:
# опять с графиками эксперементирую

# fig, ax = plt.subplots()

# ax.plot(x, y)


# ax.set_title('Промтех. ОДДС план - факт')
# ax.legend(loc='upper left')
# ax.set_xlabel('тыс.руб.')
# ax.set_ylabel('Статьи ДДС')
# fig.tight_layout()
# fig.set_figwidth(12)    #  ширина и
# fig.set_figheight(14)    #  высота "Figure"

# fig.set_facecolor('floralwhite')
# ax.set_facecolor('seashell')

# plt.show()

In [300]:
# pt1 = pd.pivot_table(m1,index = ['Статья2'], values = ['План', 'Факт', 'абс.откл.', 'отн.откл.%'], aggfunc=np.sum, fill_value='', margins=True) # np.mean

# pt1

# какая то еружнда в суммировании по столбцу факт..

In [301]:
# pt2 = pd.pivot_table(m1,index = ['Статья2'], values = ['Факт', 'План'], aggfunc=np.sum, fill_value='', margins=True) # np.mean

# pt2

In [302]:
# занимаюсь с табл с YM

# тут reset_index делать не надо
# pt_f2 = pt_f2.reset_index()
# pt_b2 = pt_b2.reset_index()

# pt_f2

In [303]:
m3 = pt_b2.merge(pt_f2, how = 'outer', on='Группа')
# m3.reset_index()

m3.fillna(0)



План                                \
YM                                     2021-01   2021-02   2021-03   2021-04   
Группа                                                                         
                                          0.00      0.00      0.00      0.00   
1. Приток от опер-ной деятельности    25524.60  79054.00  97635.84  76879.27   
2.1 Расходы на персонал              -13981.70 -18414.00 -20894.24 -21538.80   
2.2 Материалы                        -10497.05  -7670.00  -6708.76  -5214.59   
2.3 Работы и услуги                   -1216.55  -4103.00  -8187.90 -11802.53   
2.4 Страхование                                            -850.00      0.00   
2.5 Налоги, сборы и платежи           -2152.50  -1621.00 -31227.00 -14090.00   
2.6 Прочие платежи                    -6211.10 -10298.00  -8868.00  -9368.00   
3. Приток от инвест-ной деятельности   3519.80   1365.00  17600.00  26318.00   
4. Отток от инвест-ной деятельности    -431.30  -7361.00 -52980.24 -37042.38   
5. Приток от фин-ной деятельности     17012.90                0.00      0.00   
6. Отток от фин-ной деятельности      -2500.00  -4037.00 -20225.00  -2451.51   
7. Внутренние перемещения                    0         0      0.00      0.00   
9. Возврат                                   0         0      0.00      0.00   

                                                                              \
YM                                     2021-05   2021-06   2021-07   2021-08   
Группа                                                                         
                                          0.00      0.00      0.00      0.00   
1. Приток от опер-ной деятельности    70685.47  49782.55  59740.47  48820.47   
2.1 Расходы на персонал              -19640.43 -18550.74 -17917.05 -18008.40   
2.2 Материалы                         -4944.79  -4645.30  -4579.18  -4983.39   
2.3 Работы и услуги                  -18148.15 -20488.15 -18064.30 -15365.10   
2.4 Страхование                           0.00    -50.00      0.00      0.00   
2.5 Налоги, сборы и платежи           -7700.00  -5500.00  -5840.00  -5450.00   
2.6 Прочие платежи                   -10628.00 -10628.00 -16038.00 -10678.00   
3. Приток от инвест-ной деятельности   5675.00   4100.00   3500.00   3200.00   
4. Отток от инвест-ной деятельности  -30218.14 -41486.00 -23585.00 -23540.00   
5. Приток от фин-ной деятельности     40000.00  50000.00  20000.00  26000.00   
6. Отток от фин-ной деятельности      -3723.35  -1352.29  -1635.85  -5411.66   
7. Внутренние перемещения                 0.00      0.00      0.00      0.00   
9. Возврат                                0.00      0.00      0.00      0.00   

                                                              Факт            \
YM                                     2021-09   2021-10   2021-01   2021-02   
Группа                                                                         
                                          0.00      0.00         0         0   
1. Приток от опер-ной деятельности    39693.75  31750.23  25463.82  67131.00   
2.1 Расходы на персонал              -18008.40 -18133.68 -13981.78 -18359.88   
2.2 Материалы                         -5559.64  -6348.27 -10496.98  -7670.10   
2.3 Работы и услуги                  -11226.60  -4330.35  -1216.59  -4102.68   
2.4 Страхование                           0.00      0.00                       
2.5 Налоги, сборы и платежи           -3950.00  -4990.00  -2152.70  -1637.33   
2.6 Прочие платежи                   -10678.00  -9368.00  -6213.31 -10287.82   
3. Приток от инвест-ной деятельности   3100.00   2500.00   3580.78  13288.14   
4. Отток от инвест-ной деятельности  -32454.00 -17115.00   -430.83  -7360.48   
5. Приток от фин-ной деятельности     45000.00  30000.00  17012.63             
6. Отток от фин-ной деятельности      -5704.44  -2987.23  -2500.00  -4036.57   
7. Внутренние перемещения                 0.00      0.00      0.00      0.00   
9. Возврат                              

In [304]:
# m4 = pt_b2.merge(pt_f2, how = 'outer', left_on = 'Статья2', right_on = 'Статья2') # это тоже самое

# m4

In [305]:
txt_today = f"Отчет создан: {today2}"

txt_today

'Отчет создан: 2021-10-20'

In [306]:
def write_to_html_file_2(df, title='', filename='out.html'):
    '''
    Write an entire dataframe to an HTML file with nice formatting.
    '''

    result = '''
<html>
<head>
<style>

    h2 {
    text-align: center;
    font-family: Helvetica, Arial, sans-serif;
    color: #1919bc;
  }
  table { 
    margin-left: auto;
    margin-right: auto;
  }
  table, th, td {
    border: 0.5px solid black;
    border-collapse: collapse;
    border-color: #1919bc;
  }
  th, td {
    padding: 5px;
    font-family: Helvetica, Arial, sans-serif;
    font-size: 80%;
    text-align: left;
    color: #1919bc;
    vertical-align: middle;
  }
  table tbody tr:hover {
    background-color: #dddddd;
  }
  .wide {
    width: 90%;
  }

  thead > tr:nth-child(2) th:not(:first-child) {
    text-align: center;
  }
  tbody td {
    text-align: right;
  }



</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    result += df.to_html(escape=False) #classes='wide', float_format='{:.2f}'.format
    result += '''

<script>
  function numberWithSpaces(x) {
    var rounded = Math.floor(Number(x));
    var parts = rounded.toString().split(".");
    parts[0] = parts[0].replace(/\B(?=(\d{3})+(?!\d))/g, " ");
    return parts.join(".");
  }

  var items = document.querySelectorAll('tbody td');

  items.forEach(function(item) {
    var formattedNumber = numberWithSpaces(item.textContent);
    if(item.textContent === '') {
      return;
    }
    item.textContent = formattedNumber;
  });
</script>


</body>

<center>
	<footer>
      <br />
      <br />
      <b>Создал:</b> <i>Смоленышев Олег В.</i>
      <br />
      <a href="mailto:smolenyshev@gtm-one.com">smolenyshev@gtm-one.com</a>
  </footer>
</center>


</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

In [307]:
#текст для заголовков с датой
txt1 = f"Промтех. ОДДС за 2021 год (тыс.руб.) по группам и статьям ДС.  Данные из 1С ERP. Отчет на дату: {lastday2}"
txt2 = f"Промтех. ОДДС за 2021 год (тыс.руб.) только по группам ДС.  Данные из 1С ERP. Отчет на дату: {lastday2}"
txt3 = f"Промтех. Приток ден.средств по опер-ной и инвест-ной деятельности в разрезе контрагенов за 2021 год (тыс.руб.). Данные из 1С ERP. Отчет на дату: {lastday2}"
txt4 = f"Промтех. ОДДС за по дням за текущий ({tm}) месяц. (тыс.руб.) Данные из 1С ERP. Отчет на дату: {lastday2}"
txt5 = f"Промтех. Приток ден.средств по опер-ной и инвест-ной деятельности в разрезе контрагенов за текущий ({tm}) месяц. (тыс.руб.) Данные из 1С ERP. Отчет на дату: {lastday2}"



text1 = f"Промтех. План-факт движения ден.средств в целом с января 2021 по текущий месяц. Факт из 1С ERP. План из бюджета представленного к защите 24 марта 2021. Актуализация на дату: {lastday2}. (тыс.руб.)"
text2 = f"Промтех. План-факт движения ден.средств по месяцам с января 2021 по текущий месяц. Факт из 1С ERP. План из бюджета представленного к защите 24 марта 2021. Актуализация на дату: {lastday2}. (тыс.руб.)"

txt5

"Промтех. Приток ден.средств по опер-ной и инвест-ной деятельности в разрезе контрагенов за текущий (10) месяц. (тыс.руб.) Данные из 1С ERP. Отчет на дату: ['2021-10-18']"

In [308]:
text3 = f"Промтех. План-факт движения ден.средств в целом с января 2021 по текущий месяц. Факт из 1С ERP. \n План из бюджета представленного к защите 24 марта 2021. Актуализация на дату: {lastday2}. \n \
Всего чистый суммарный приток ден.средств по плану: {plan} тыс.руб., по факту: {fact} тыс.руб., абс.откл.: {abs} тыс.руб., отн.отклонение: {otn}% \n \
Приток ден средств по опер.деят-ти за текущий ({ty}-{tm}) месяц составил: {pritok_tm} тыс.руб. \n \
Отток ден средств по опер.деят-ти за текущий ({ty}-{tm}) месяц составил: {ottok_tm} тыс.руб."

print(text3)

Промтех. План-факт движения ден.средств в целом с января 2021 по текущий месяц. Факт из 1С ERP. 
 План из бюджета представленного к защите 24 марта 2021. Актуализация на дату: ['2021-10-18']. 
 Всего чистый суммарный приток ден.средств по плану: 16861.3 тыс.руб., по факту: 104635.5 тыс.руб., абс.откл.: 87774.2 тыс.руб., отн.отклонение: 520.57% 
 Приток ден средств по опер.деят-ти за текущий (2021-10) месяц составил: 35407.8 тыс.руб. 
 Отток ден средств по опер.деят-ти за текущий (2021-10) месяц составил: -31775.0 тыс.руб.


In [309]:
# создание html файлов
write_to_html_file_2(pt1, txt1, 'Промтех_ОДДС_2021.html')
write_to_html_file_2(pt2, txt2, 'Промтех_ОДДС_2021_кратк.html')
write_to_html_file_2(pt3, txt3, 'Промтех_ОДДС_приток_2021.html')
write_to_html_file_2(pt4, txt4, 'Промтех_ОДДС_2021_текущ_мес.html')
write_to_html_file_2(pt5, txt5, 'Промтех_ОДДС_приток_2021_текущ_мес.html')

write_to_html_file_2(m2, text3, 'Промтех_ОДДС_план-факт_2021.html')
write_to_html_file_2(m3, text2, 'Промтех_ОДДС_план-факт_2021_текущ_мес.html')

In [310]:
#Сохранение всех сводных табл в excel файл
# writer = pd.ExcelWriter('pt_odds_v15.xlsx', engine='xlsxwriter')

# df.to_excel(writer, sheet_name='df', header = True)
# df_b.to_excel(writer, sheet_name='df_b', header = True)

# pt_f1.to_excel(writer, sheet_name='pt_f1', header = True)
# pt_b1.to_excel(writer, sheet_name='pt_b1', header = True)

# pt_f2.to_excel(writer, sheet_name='pt_f2', header = True)
# pt_b2.to_excel(writer, sheet_name='pt_b2', header = True)

# m1.to_excel(writer, sheet_name='m1', header = True)
# m2.to_excel(writer, sheet_name='m2', header = True)

# pt1.to_excel(writer, sheet_name='pt1', header = True)


# writer.save()
# writer.close()

In [311]:
# Отправка на почту

import smtplib
# import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

In [320]:
# Настройки

sender_email = 'syslog2@vprom.tech'
#receiver_email = 'smolenyshev@gtm-one.com, smolenishev@mail.ru, sitnikov_sp@vprom.tech'
receiver_email = 'smolenyshev@gtm-one.com'
# receiver_email = 'smolenyshev@gtm-one.com, Kasimov@gtm-one.com, Haibulin@gtm-one.com, Idrisov@gtm-one.com, Kolesnikova_LN@vProm.Tech, Nazarova_VV@vProm.Tech, nikulina_ma@vprom.tech' 

username = 'vprom\\syslog2'
password = 'j0HosyKy3FHp'
server = smtplib.SMTP('mail.vprom.tech:587')
subject = f'Промтех. ОДДС. Отчет на дату: {lastday2}' #тема пиьма

# body = f"Добрый день.\n\nПередаю для инф. ОДДС. Отчет на дату: {lastday2}. \nОтчет сформирован на основании данных из 1С ERP. \
#          С 9 сентября 2021 отчет запускается каждый рабочий день в 08:10.\n С 21 октября 2021 отчеты изменены.\n\n \
#          С уважением, Смоленышев О.В."  #текст внутри пиьма, если нужен

body = f"Доброе утро. \n\n Передаю для информации ОДДС фактическое движение и план-факт.\n \
        Отчеты формируются на основании данных из 1С ERP и и представленнго к защите бюджета от 24 марта 2021. Отчет на дату: {lastday2}. \n \
        С 9 сентября 2021 отчет запускается каждый рабочий день в 08:10. \n\n \
        Всего чистый суммарный приток ден.средств по плану: {plan} тыс.руб., по факту: {fact} тыс.руб., \n \
        абс.откл.: {abs} тыс.руб., отн.отклонение: {otn}%  \n\n \
        Приток ден средств по опер.деят-ти за текущий ({ty}-{tm}) месяц составил: {pritok_tm} тыс.руб. \n \
        Отток ден средств по опер.деят-ти за текущий ({ty}-{tm}) месяц составил: {ottok_tm} тыс.руб. \n\n \
        С 21 октября 2021 отчеты адоптированы. \n\
        С уважением, Смоленышев О.В."


dir_path = "C:\\Reports"  #путь к файлам

files = ['Промтех_ОДДС_2021.html', 'Промтех_ОДДС_2021_кратк.html', 'Промтех_ОДДС_приток_2021.html', 'Промтех_ОДДС_2021_текущ_мес.html', 
        'Промтех_ОДДС_приток_2021_текущ_мес.html', 'Промтех_ОДДС_план-факт_2021_текущ_мес.html']        #список отправляемых файлов. , 'cf_graf.html'

In [321]:
# Создание составного сообщения и установка заголовка

message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject

# Внесение тела письма
message.attach(MIMEText(body, "plain"))

for f in files:
    file_path = os.path.join(dir_path,f)
    attachment = MIMEApplication(open(file_path, "rb").read(), _subtype="txt")
    attachment.add_header('Content-Disposition', 'attachment', filename=f)
    message.attach(attachment)

# Отпавляем письмо

server.starttls()
server.ehlo()
server.login(username, password)
server.sendmail(message['From'], message['To'].split(','), message.as_string())
server.quit()
print(receiver_email, files, sep='\n')

smolenyshev@gtm-one.com
['Промтех_ОДДС_2021.html', 'Промтех_ОДДС_2021_кратк.html', 'Промтех_ОДДС_приток_2021.html', 'Промтех_ОДДС_2021_текущ_мес.html', 'Промтех_ОДДС_приток_2021_текущ_мес.html', 'Промтех_ОДДС_план-факт_2021_текущ_мес.html']
